# Classification - Azure AI Document Intelligence + Embeddings

This sample demonstrates how to classify a document using Azure AI Document Intelligence and text embeddings.

![Data Classification](../../images/classification-embeddings.png)

This is achieved by the following process:

- Define a list of classifications, with descriptions and keywords.
- Create text embeddings for each of the classifications.
- Analyze a document using Azure AI Document Intelligence's `prebuilt-layout` model to extract the text from each page.
- For each page:
  - Create text embeddings.
  - Compare the embeddings with the embeddings of each classification.
  - Assign the page to the classification with the highest similarity that exceeds a given threshold.

## Objectives

By the end of this sample, you will have learned how to:

- Convert text to embeddings using Azure OpenAI's `text-embedding-3-large` model.
- Convert a document's pages to Markdown format using Azure AI Document Intelligence.
- Use cosine similarity to compare embeddings of classifications with document pages to classify them.

## Setup

### Import modules

This sample takes advantage of the following Python dependencies:

- **numpy** and **sklearn** for determining the cosine similarity between embeddings.
- **azure-ai-documentintelligence** to interface with the Azure AI Document Intelligence API for analyzing documents.
- **openai** to interface with the Azure OpenAI API for generating text embeddings.
- **azure-identity** to securely authenticate with deployed Azure Services using Microsoft Entra ID credentials.

The following local modules are also used:

- **modules.app_settings** to access environment variables from the `.env` file.
- **modules.classification** to define the classifications.
- **modules.comparison** to compare the output of the classification process with expected results.
- **modules.document_processing_result** to store the results of the classification process as a file.
- **modules.stopwatch** to measure the end-to-end execution time for the classification process.

In [1]:
import sys
sys.path.append('../') # Import local modules

from IPython.display import display, Markdown
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os
import pandas as pd
from dotenv import dotenv_values
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, ContentFormat
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

from modules.app_settings import AppSettings
from modules.classification import Classifications, Classification
from modules.comparison import classification_comparison
from modules.document_processing_result import DataClassificationResult
from modules.stopwatch import Stopwatch

### Configure the Azure services

To use Azure AI Document Intelligence and Azure OpenAI, their SDKs are used to create client instances using a deployed endpoint and authentication credentials.

For this sample, the credentials of the Azure CLI are used to authenticate with the deployed services.

In [2]:
# Set the working directory to the root of the repo
working_dir = os.path.abspath('../../')
settings = AppSettings(dotenv_values(f"{working_dir}/.env"))

# Configure the default credential for accessing Azure services using Azure CLI credentials
credential = DefaultAzureCredential(
    exclude_workload_identity_credential=True,
    exclude_developer_cli_credential=True,
    exclude_environment_credential=True,
    exclude_managed_identity_credential=True,
    exclude_powershell_credential=True,
    exclude_shared_token_cache_credential=True,
    exclude_interactive_browser_credential=True
)

openai_token_provider = get_bearer_token_provider(credential, 'https://cognitiveservices.azure.com/.default')

openai_client = AzureOpenAI(
    azure_endpoint=settings.openai_endpoint,
    azure_ad_token_provider=openai_token_provider,
    api_version="2024-08-01-preview" # Requires the latest API version for structured outputs.
)

document_intelligence_client = DocumentIntelligenceClient(
    endpoint=settings.ai_services_endpoint,
    credential=credential
)

### Establish the expected output

To compare the accuracy of the classification process, the expected output of the classification process has been defined in the following code block based on each page of the [Vehicle Insurance Policy](../assets/VehicleInsurancePolicy.pdf).

The expected output has been defined by a human evaluating the document.

> **Note**: Only the `page_number` and `classification` are used in the accuracy evaluation.

In [3]:
pdf_path = f"{working_dir}/samples/assets/"
pdf_file_name = "VehicleInsurancePolicy.pdf"
fname = f"{pdf_path}{pdf_file_name}"

expected = Classifications(classifications=[
    Classification(page_number=0, classification="Insurance Policy", similarity=1, all_similarities=[]),
    Classification(page_number=1, classification="Insurance Policy", similarity=1, all_similarities=[]),
    Classification(page_number=2, classification="Insurance Policy", similarity=1, all_similarities=[]),
    Classification(page_number=3, classification="Insurance Policy", similarity=1, all_similarities=[]),
    Classification(page_number=4, classification="Insurance Policy", similarity=1, all_similarities=[]),
    Classification(page_number=5, classification="Insurance Certificate", similarity=1, all_similarities=[]),
    Classification(page_number=6, classification="Terms and Conditions", similarity=1, all_similarities=[]),
    Classification(page_number=7, classification="Terms and Conditions", similarity=1, all_similarities=[]),
    Classification(page_number=8, classification="Terms and Conditions", similarity=1, all_similarities=[]),
    Classification(page_number=9, classification="Terms and Conditions", similarity=1, all_similarities=[]),
    Classification(page_number=10, classification="Terms and Conditions", similarity=1, all_similarities=[]),
    Classification(page_number=11, classification="Terms and Conditions", similarity=1, all_similarities=[]),
    Classification(page_number=12, classification="Terms and Conditions", similarity=1, all_similarities=[])
])

## Define classifications

The following code block defines the classifications for a document. Each classification has a name, description, and keywords that will be used to generate embeddings and compare similarity with each page of the document.

> **Note**, the classifications have been defined based on expected content in a specific type of document, in this example, [the Vehicle Insurance Policy](../assets/VehicleInsurancePolicy.pdf).

In [4]:
classifications = [
    {
        "classification": "Insurance Policy",
        "description": "Specific information related to an insurance policy, such as coverage, limits, premiums, and terms, often used for reference or clarification purposes.",
        "keywords": [
            "cover type",
            "effective date",
            "expiry date",
            "excesses",
            "premium",
            "policy number",
        ]
    },
    {
        "classification": "Insurance Certificate",
        "description": "A document that serves as proof of insurance coverage, often required for legal, regulatory, or contractual purposes.",
        "keywords": [
            "certificate",
            "proof",
            "coverage",
            "liability",
            "endorsement",
            "declaration",
        ]
    },
    {
        "classification": "Terms and Conditions",
        "description": "The rules, requirements, or obligations that govern an agreement or contract, often related to insurance policies, financial products, or legal documents.",
        "keywords": [
            "legal",
            "statements",
            "terms",
            "conditions",
            "rules",
            "requirements",
            "obligations",
            "agreement",
            "responsibilities",
            "payment",
            "renewal",
            "cancellation",
            "what's covered",
            "what's not covered",
            "exclusions",
            "limitations",
            "exceptions",
            "restrictions",
        ]
    }
]

## Convert the document pages to Markdown

To classify the document pages using embeddings, the text from each page must first be extracted.

The following code block converts the document pages to Markdown format using Azure AI Document Intelligence's `prebuilt-layout` model.

For the purposes of this sample, we will be classifying each page. The benefit of using Azure AI Document Intelligence for this extraction is that it provides a page-by-page analysis result of the document.

In [5]:
with Stopwatch() as di_stopwatch:
    with open(fname, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-layout",
            analyze_request=f,
            output_content_format=ContentFormat.MARKDOWN,
            content_type="application/pdf"
        )
        
    result: AnalyzeResult = poller.result()

pages_content = []
for page in result.pages:
    # Extract the entire content for each page of the document based on the span offsets and lengths
    content = result.content[page.spans[0]['offset']: page.spans[0]['offset'] + page.spans[0]['length']]
    pages_content.append(content)

## Create embeddings

With the text extracted from the document and the classifications defined, the next step is to create embeddings for each page and classification.

### Retrieving embeddings for text

The following helper function retrieves embeddings for a given piece of text using Azure OpenAI's `text-embedding-3-large` model.

In [6]:
def get_embedding(text: str):
    response = openai_client.embeddings.create(
        input=text,
        model=settings.text_embedding_model_deployment_name
    )
    embedding = response.data[0].embedding
    return embedding

### Convert the classifications to embeddings

The following code block takes each classification and generates the embeddings for the keywords.

In [7]:
with Stopwatch() as ce_stopwatch:
    for classification in classifications:
        combined_text = f"{' '.join(classification['keywords'])}"
        classification['embedding'] = get_embedding(combined_text)

### Convert the document pages to embeddings

The following code block takes each page of the document and generates the embeddings for the text.

In [8]:
with Stopwatch() as de_stopwatch:
    page_embeddings = []
    for idx, text in enumerate(pages_content):
        embedding = get_embedding(text)
        page_embeddings.append(embedding)

## Classify the document pages

The following code block runs the classification process using cosine similarity to compare the embeddings of the document pages with the embeddings of the predefined categories.

It performs the following steps iteratively for each page in the document:

1. Calculates the cosine similarity between the embeddings of the page and the matrix of embeddings of the predefined categories.
2. Finds the best match for the page based on the maximum cosine similarity score.
3. If the cosine similarity score is above a certain threshold, the page is classified under the best match category. Otherwise, the page is classified as "Unclassified".

In [9]:
similarity_threshold = 0.4

classification_embeddings = [cls['embedding'] for cls in classifications]
classification_matrix = np.array(classification_embeddings)

with Stopwatch() as classify_stopwatch:
    document_classifications = Classifications(classifications=[])
    for idx, page_emb in enumerate(page_embeddings):
        if not page_emb:
            classification = "Unclassified"
            similarity = 0.0
        else:
            page_vector = np.array(page_emb).reshape(1, -1)
            similarities = cosine_similarity(page_vector, classification_matrix)[0]
            best_match_idx = np.argmax(similarities)
            best_similarity = similarities[best_match_idx]
            all_similarities: list[dict[str,str]] = []
            for cls, sim in zip(classifications, similarities):
                all_similarities.append({
                    "classification": cls['classification'],
                    "similarity": str(sim)
                })

            if best_similarity >= similarity_threshold:
                classification = classifications[best_match_idx]['classification']
            else:
                classification = "Unclassified"
                
        document_classifications.classifications.append(
            Classification(
                page_number=idx,
                classification=classification,
                similarity=best_similarity,
                all_similarities=all_similarities
            )
        )

## Calculate the accuracy

The following code block calculates the accuracy of the classification process by comparing the actual classifications with the predicted classifications.

In [10]:
accuracy = dict()

for idx, classification in enumerate(document_classifications.classifications):
    page_number = classification.page_number
    expected_classification = expected.get_classification(page_number).classification
    actual_classification = classification.classification
    accuracy[page_number] = 1 if expected_classification == actual_classification else 0
    
overall = sum(accuracy.values()) / len(accuracy)
accuracy['overall'] = overall

## Visualize the outputs

To provide context for the execution of the code, the following code blocks visualize the outputs of the classification process.

This includes:

- The accuracy of the classification process comparing the expected output with the result of comparing the embeddings.
- The execution time of the end-to-end process.
- The classification results for each page in the document.

### Understanding Similarity

Cosine similarity is a metric used to measure how similar two vectors are. Embeddings are numerical representations of text. By converting a document page and classification keywords to embeddings, we can compare the similarity between the two using this technique.

Similarity scores close to 1 indicate that the two vectors share similar characteristics, while scores closer to 0 or negative values indicate that the two vectors are dissimilar.

In [11]:
# Gets the total execution time of the classification process.
total_elapsed = di_stopwatch.elapsed + ce_stopwatch.elapsed + de_stopwatch.elapsed + classify_stopwatch.elapsed

# Save the output of the data classification result.
classification_result = DataClassificationResult(
    classification=document_classifications.to_dict(),
    accuracy=accuracy,
    execution_time=total_elapsed
)

with open(f"{working_dir}/samples/classification/document-intelligence-embeddings.{pdf_file_name}.json", "w") as f:
    f.write(classification_result.to_json(indent=4))

# Display the outputs of the classification process.
df = pd.DataFrame([
    {
        "Accuracy": f"{accuracy['overall'] * 100:.2f}%",
        "Execution Time": f"{total_elapsed:.2f} seconds",
        "Document Intelligence Execution Time": f"{di_stopwatch.elapsed:.2f} seconds",
        "Classification Embedding Execution Time": f"{ce_stopwatch.elapsed:.2f} seconds",
        "Document Embedding Execution Time": f"{de_stopwatch.elapsed:.2f} seconds",
        "Classification Execution Time": f"{classify_stopwatch.elapsed:.2f} seconds"
    }
])

display(Markdown(df.to_markdown(index=False, tablefmt='unsafehtml')))
display(Markdown(classification_comparison(expected, document_classifications)))

<table>
<thead>
<tr><th>Accuracy  </th><th>Execution Time  </th><th>Document Intelligence Execution Time  </th><th>Classification Embedding Execution Time  </th><th>Document Embedding Execution Time  </th><th>Classification Execution Time  </th></tr>
</thead>
<tbody>
<tr><td>92.31%    </td><td>11.17 seconds   </td><td>7.41 seconds                          </td><td>1.70 seconds                             </td><td>2.02 seconds                       </td><td>0.04 seconds                   </td></tr>
</tbody>
</table>

<table>
<thead>
<tr><th style="text-align: right;">  Page</th><th>Expected             </th><th>Extracted                                              </th><th style="text-align: right;">  Similarity</th><th>Matches                                                                                                                                                        </th></tr>
</thead>
<tbody>
<tr><td style="text-align: right;">     0</td><td>Insurance Policy     </td><td><span style='color: green'>Insurance Policy</span>     </td><td style="text-align: right;">    0.499674</td><td><ul><li>Insurance Policy (0.49967448413024956)</li><li>Insurance Certificate (0.42930679934653765)</li><li>Terms and Conditions (0.37564145464816273)</li></ul></td></tr>
<tr><td style="text-align: right;">     1</td><td>Insurance Policy     </td><td><span style='color: green'>Insurance Policy</span>     </td><td style="text-align: right;">    0.492551</td><td><ul><li>Insurance Policy (0.492550682114909)</li><li>Insurance Certificate (0.43398913025203145)</li><li>Terms and Conditions (0.404896770185204)</li></ul>    </td></tr>
<tr><td style="text-align: right;">     2</td><td>Insurance Policy     </td><td><span style='color: green'>Insurance Policy</span>     </td><td style="text-align: right;">    0.478154</td><td><ul><li>Insurance Policy (0.4781539768108446)</li><li>Insurance Certificate (0.4355942432519841)</li><li>Terms and Conditions (0.3689413542285723)</li></ul>   </td></tr>
<tr><td style="text-align: right;">     3</td><td>Insurance Policy     </td><td><span style='color: red'>Unclassified</span>           </td><td style="text-align: right;">    0.39564 </td><td><ul><li>Insurance Policy (0.39564022820051725)</li><li>Terms and Conditions (0.35477255739672475)</li><li>Insurance Certificate (0.34029757817106654)</li></ul></td></tr>
<tr><td style="text-align: right;">     4</td><td>Insurance Policy     </td><td><span style='color: green'>Insurance Policy</span>     </td><td style="text-align: right;">    0.602353</td><td><ul><li>Insurance Policy (0.6023526579770556)</li><li>Insurance Certificate (0.43507389038041244)</li><li>Terms and Conditions (0.43269067427502883)</li></ul> </td></tr>
<tr><td style="text-align: right;">     5</td><td>Insurance Certificate</td><td><span style='color: green'>Insurance Certificate</span></td><td style="text-align: right;">    0.526395</td><td><ul><li>Insurance Certificate (0.5263946839733032)</li><li>Insurance Policy (0.5116703812992802)</li><li>Terms and Conditions (0.4119173918990154)</li></ul>   </td></tr>
<tr><td style="text-align: right;">     6</td><td>Terms and Conditions </td><td><span style='color: green'>Terms and Conditions</span> </td><td style="text-align: right;">    0.512304</td><td><ul><li>Terms and Conditions (0.5123037092284444)</li><li>Insurance Policy (0.4364133997063372)</li><li>Insurance Certificate (0.40406005318387517)</li></ul>  </td></tr>
<tr><td style="text-align: right;">     7</td><td>Terms and Conditions </td><td><span style='color: green'>Terms and Conditions</span> </td><td style="text-align: right;">    0.529807</td><td><ul><li>Terms and Conditions (0.529807463777394)</li><li>Insurance Policy (0.47225913160981425)</li><li>Insurance Certificate (0.4313313331404175)</li></ul>   </td></tr>
<tr><td style="text-align: right;">     8</td><td>Terms and Conditions </td><td><span style='color: green'>Terms and Conditions</span> </td><td style="text-align: right;">    0.467829</td><td><ul><li>Terms and Conditions (0.46782884412542797)</li><li>Insurance Policy (0.42621188856260067)</li><li>Insurance Certificate (0.37135628647491326)</li></ul></td></tr>
<tr><td style="text-align: right;">     9</td><td>Terms and Conditions </td><td><span style='color: green'>Terms and Conditions</span> </td><td style="text-align: right;">    0.481846</td><td><ul><li>Terms and Conditions (0.481846340542209)</li><li>Insurance Certificate (0.440268346375272)</li><li>Insurance Policy (0.42257533753316334)</li></ul>    </td></tr>
<tr><td style="text-align: right;">    10</td><td>Terms and Conditions </td><td><span style='color: green'>Terms and Conditions</span> </td><td style="text-align: right;">    0.479737</td><td><ul><li>Terms and Conditions (0.47973712095188187)</li><li>Insurance Policy (0.43987104749822503)</li><li>Insurance Certificate (0.4188467103428988)</li></ul> </td></tr>
<tr><td style="text-align: right;">    11</td><td>Terms and Conditions </td><td><span style='color: green'>Terms and Conditions</span> </td><td style="text-align: right;">    0.501895</td><td><ul><li>Terms and Conditions (0.5018953040109185)</li><li>Insurance Policy (0.4833041785380461)</li><li>Insurance Certificate (0.43905074983717396)</li></ul>  </td></tr>
<tr><td style="text-align: right;">    12</td><td>Terms and Conditions </td><td><span style='color: green'>Terms and Conditions</span> </td><td style="text-align: right;">    0.546613</td><td><ul><li>Terms and Conditions (0.5466127859591476)</li><li>Insurance Policy (0.47031919426961744)</li><li>Insurance Certificate (0.4167836286559835)</li></ul>  </td></tr>
</tbody>
</table>